Import section

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Read files

In [2]:
car_train=pd.read_csv("../data/train.csv")
car_test=pd.read_csv("../data/test.csv")

In [3]:
car_train.shape

(4819, 14)

In [4]:
car_test.shape

(1159, 13)

In [5]:
car_train.head()

,id,Nombre,Ciudad,Año,Kilometros,Combustible,Tipo_marchas,Mano,Consumo,Motor_CC,Potencia,Asientos,Descuento,Precio_cat
0,1.0,Maruti Swift Dzire ZDI,G,2012.0,83000.0,Diesel,Manual,First,23.4 kmpl,1248 CC,74 bhp,5.0,NaN,3
1,2.0,Maruti Wagon R LXI Optional,I,2016.0,4800.0,Petrol,Manual,First,20.51 kmpl,998 CC,67.04 bhp,5.0,NaN,3
2,3.0,Mahindra KUV 100 mFALCON D75 K8,F,2016.0,26000.0,Diesel,Manual,First,25.32 kmpl,1198 CC,77 bhp,6.0,NaN,3
3,4.0,Hyundai i20 1.2 Magna,E,2013.0,56127.0,Petrol,Manual,First,18.5 kmpl,1197 CC,80 bhp,5.0,NaN,3
4,5.0,Honda Jazz 1.2 SV i VTEC,H,2017.0,41981.0,Petrol,Manual,First,18.7 kmpl,1199 CC,88.7 bhp,5.0,NaN,3


In [50]:
car_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4001 entries, 0 to 4818
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Nombre        4001 non-null   int64  
 1   Ciudad        4001 non-null   int64  
 2   Año           4001 non-null   float64
 3   Kilometros    4001 non-null   float64
 4   Combustible   4001 non-null   int64  
 5   Tipo_marchas  4001 non-null   int64  
 6   Mano          4001 non-null   int64  
 7   Consumo       4001 non-null   int64  
 8   Motor_CC      4001 non-null   int64  
 9   Potencia      4001 non-null   int64  
 10  Asientos      4001 non-null   float64
 11  Descuento     4001 non-null   float64
 12  Precio_cat    4001 non-null   int64  
dtypes: float64(4), int64(9)
memory usage: 437.6 KB


In [7]:
car_test.head()

,id,Nombre,Ciudad,Año,Kilometros,Combustible,Tipo_marchas,Mano,Consumo,Motor_CC,Potencia,Asientos,Descuento
0,4820,Mercedes-Benz New C-Class C 220 CDI Avantgarde,H,2015,77771,Diesel,Automatic,First,19.27 kmpl,2143 CC,170 bhp,5.0,NaN
1,4821,Hyundai i10 Magna 1.1,J,2011,62000,Petrol,Manual,First,19.81 kmpl,1086 CC,68.05 bhp,5.0,NaN
2,4822,Ford Figo Diesel EXI,C,2010,129986,Diesel,Manual,Second,20.0 kmpl,1399 CC,68 bhp,5.0,NaN
3,4823,Maruti Swift Dzire VXI Optional,D,2015,49900,Petrol,Manual,First,20.85 kmpl,1197 CC,83.14 bhp,5.0,NaN
4,4824,Maruti Ritz VXI,J,2010,75319,Petrol,Manual,Second,21.1 kmpl,1197 CC,85.80 bhp,5.0,NaN


In [51]:
car_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1159 entries, 0 to 1158
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Nombre        1159 non-null   int64  
 1   Ciudad        1159 non-null   int64  
 2   Año           1159 non-null   int64  
 3   Kilometros    1159 non-null   int64  
 4   Combustible   1159 non-null   int64  
 5   Tipo_marchas  1159 non-null   int64  
 6   Mano          1159 non-null   int64  
 7   Consumo       1159 non-null   int64  
 8   Motor_CC      1159 non-null   int64  
 9   Potencia      1159 non-null   int64  
 10  Asientos      1159 non-null   float64
 11  Descuento     1159 non-null   float64
dtypes: float64(2), int64(10)
memory usage: 108.8 KB


In [9]:
# Remove id column
car_train = car_train.drop('id', axis = 1)
tests_ids = car_test['id']
del car_test['id']

In [10]:
print("Valores únicos en el conjunto de entrenamiento:\n")
for i in car_train.columns:
    print(i, car_train[i].unique())

Valores únicos en el conjunto de entrenamiento:

Nombre ['Maruti Swift Dzire ZDI' 'Maruti Wagon R LXI Optional'
 'Mahindra KUV 100 mFALCON D75 K8' ...
 'Mercedes-Benz E-Class 280 Elegance' 'Honda City Corporate Edition'
 'Porsche Panamera Diesel 250hp']
Ciudad ['G' 'I' 'F' 'E' 'H' 'C' 'J' 'L' 'B' 'K' 'D' nan]
Año [2012. 2016. 2013. 2017. 2009. 2014. 2011. 2015. 2008. 2010. 2018. 2006.
   nan 2019. 2005. 2007. 2003. 2004. 2001. 1998. 2002. 2000. 1999.]
Kilometros [83000.  4800. 26000. ... 60033. 40158. 65743.]
Combustible ['Diesel' 'Petrol' nan 'CNG' 'LPG' 'Electric']
Tipo_marchas ['Manual' 'Automatic' nan]
Mano ['First' 'Second' 'Third' nan 'Fourth & Above']
Consumo ['23.4 kmpl' '20.51 kmpl' '25.32 kmpl' '18.5 kmpl' '18.7 kmpl' '17.8 kmpl'
 '16.8 kmpl' '18.6 kmpl' '25.2 kmpl' '10.91 kmpl' '12.99 kmpl' '17.5 kmpl'
 '25.8 kmpl' '18.25 kmpl' '10.0 kmpl' '11.5 kmpl' '19.1 kmpl' '14.21 kmpl'
 '14.16 kmpl' '17.43 kmpl' '18.0 kmpl' '15.7 kmpl' '16.0 kmpl'
 '21.43 kmpl' '21.64 kmpl' '20.73 kmp

In [11]:
print("Valores únicos en el conjunto de test:\n")
for i in car_test.columns:
    print(i, car_test[i].unique())

Valores únicos en el conjunto de test:

Nombre ['Mercedes-Benz New C-Class C 220 CDI Avantgarde' 'Hyundai i10 Magna 1.1'
 'Ford Figo Diesel EXI' 'Maruti Swift Dzire VXI Optional'
 'Maruti Ritz VXI' 'Tata Indigo LX BSII' 'Mercedes-Benz B Class B200 CDI'
 'Audi A4 2.0 TDI 177 Bhp Premium Plus' 'Honda City i-DTEC SV'
 'Toyota Innova 2.5 G (Diesel) 8 Seater'
 'Toyota Corolla Altis Diesel D4DG' 'Mahindra XUV500 W10 AWD'
 'Maruti 800 Std BSIII' 'Tata Manza Club Class Quadrajet90 LS'
 'BMW 5 Series 2003-2012 520d' 'Toyota Fortuner 2.8 4WD MT'
 'Nissan Micra XE'
 'BMW X5 2014-2019 xDrive 30d Design Pure Experience 7 Seater'
 'Toyota Fortuner 4x2 AT' 'Maruti Alto LX' 'Ford Figo Petrol Titanium'
 'Honda City 1.3 EXI' 'Maruti A-Star Lxi' 'Honda City i VTEC SV'
 'Maruti Wagon R LXI CNG' 'Hyundai Verna 1.6 SX'
 'BMW 3 Series Luxury Line' 'Mahindra Thar DI 4X4'
 'Toyota Innova 2.5 VX (Diesel) 8 Seater BS IV'
 'Hyundai Verna 1.6 SX VTVT' 'Maruti Ertiga ZDI'
 'Honda City i VTEC VX Option' 'Toyota Coro

In [12]:
# find the number of nulls in each colum
car_train.isnull().sum()

Nombre            72
Ciudad            72
Año               72
Kilometros        72
Combustible       72
Tipo_marchas      72
Mano              72
Consumo           73
Motor_CC         101
Potencia         175
Asientos         106
Descuento       4160
Precio_cat         0
dtype: int64

In [13]:
# find the number of nulls in each colum
car_test.isnull().sum()

Nombre             0
Ciudad             0
Año                0
Kilometros         0
Combustible        0
Tipo_marchas       0
Mano               0
Consumo            0
Motor_CC           0
Potencia           0
Asientos           0
Descuento       1004
dtype: int64

In [14]:
print("There is {} duplicated values in data frame".format(car_train.duplicated().sum()))

There is 8 duplicated values in data frame


In [15]:
print("There is {} duplicated values in data frame".format(car_test.duplicated().sum()))

There is 0 duplicated values in data frame


In [16]:
car_train.isnull().sum()

Nombre            72
Ciudad            72
Año               72
Kilometros        72
Combustible       72
Tipo_marchas      72
Mano              72
Consumo           73
Motor_CC         101
Potencia         175
Asientos         106
Descuento       4160
Precio_cat         0
dtype: int64

In [17]:
print("Shape of train data Before dropping any Row: ",car_train.shape)
car_train = car_train[car_train['Nombre'].notna()]
print("Shape of train data After dropping Rows with NULL values in Nombre: ",car_train.shape)
car_train = car_train[car_train['Ciudad'].notna()]
print("Shape of train data After dropping Rows with NULL values in Ciudad: ",car_train.shape)
car_train = car_train[car_train['Año'].notna()]
print("Shape of train data After dropping Rows with NULL values in Año: ",car_train.shape)
car_train = car_train[car_train['Kilometros'].notna()]
print("Shape of train data After dropping Rows with NULL values in Kilometros: ",car_train.shape)
car_train = car_train[car_train['Combustible'].notna()]
print("Shape of train data After dropping Rows with NULL values in Combustible: ",car_train.shape)
car_train = car_train[car_train['Tipo_marchas'].notna()]
print("Shape of train data After dropping Rows with NULL values in Tipo_marchas: ",car_train.shape)
car_train = car_train[car_train['Mano'].notna()]
print("Shape of train data After dropping Rows with NULL values in Mano: ",car_train.shape)
car_train = car_train[car_train['Consumo'].notna()]
print("Shape of train data After dropping Rows with NULL values in Consumo: ",car_train.shape)
car_train = car_train[car_train['Motor_CC'].notna()]
print("Shape of train data After dropping Rows with NULL values in Motor_CC: ",car_train.shape)
car_train = car_train[car_train['Potencia'].notna()]
print("Shape of train data After dropping Rows with NULL values in Potencia: ",car_train.shape)
car_train = car_train[car_train['Asientos'].notna()]
print("Shape of train data After dropping Rows with NULL values in Asientos: ",car_train.shape)

Shape of train data Before dropping any Row:  (4819, 13)
Shape of train data After dropping Rows with NULL values in Nombre:  (4747, 13)
Shape of train data After dropping Rows with NULL values in Ciudad:  (4676, 13)
Shape of train data After dropping Rows with NULL values in Año:  (4606, 13)
Shape of train data After dropping Rows with NULL values in Kilometros:  (4540, 13)
Shape of train data After dropping Rows with NULL values in Combustible:  (4475, 13)
Shape of train data After dropping Rows with NULL values in Tipo_marchas:  (4410, 13)
Shape of train data After dropping Rows with NULL values in Mano:  (4342, 13)
Shape of train data After dropping Rows with NULL values in Consumo:  (4274, 13)
Shape of train data After dropping Rows with NULL values in Motor_CC:  (4185, 13)
Shape of train data After dropping Rows with NULL values in Potencia:  (4060, 13)
Shape of train data After dropping Rows with NULL values in Asientos:  (4001, 13)


In [18]:
car_train['Descuento']=car_train['Descuento'].fillna((car_train['Descuento'].mode()[0]))

In [19]:
car_test['Descuento']=car_test['Descuento'].fillna((car_test['Descuento'].mode()[0]))

In [20]:
car_train.isnull().sum()

Nombre          0
Ciudad          0
Año             0
Kilometros      0
Combustible     0
Tipo_marchas    0
Mano            0
Consumo         0
Motor_CC        0
Potencia        0
Asientos        0
Descuento       0
Precio_cat      0
dtype: int64

In [21]:
car_test.isnull().sum()

Nombre          0
Ciudad          0
Año             0
Kilometros      0
Combustible     0
Tipo_marchas    0
Mano            0
Consumo         0
Motor_CC        0
Potencia        0
Asientos        0
Descuento       0
dtype: int64

In [22]:
car_train.shape

(4001, 13)

In [23]:
car_test.shape

(1159, 12)

In [24]:
# Aplicando etiqueta a varios ficheros
from sklearn.preprocessing import LabelEncoder

In [25]:
df_train_orig = car_train.copy()
df_test_orig = car_test.copy()

In [26]:
for i in list(car_train.columns):
    if car_train[i].dtypes == "object":
        to_read = "../data/" + i.lower() + ".csv"
        label = LabelEncoder().fit(pd.read_csv(to_read)[i])
        car_train[i] = label.transform(df_train_orig[i])
        car_test[i] = label.transform(df_test_orig[i])

In [27]:
correcto = True

for asientos in df_train_orig.Asientos.unique():
    label_train = car_train.Asientos[df_train_orig.Asientos == asientos].unique()
    label_test = car_test.Asientos[df_test_orig.Asientos == asientos].unique()
    
    if len(label_train)==0 or len(label_test)==0:
        continue
    
    if label_train != label_test:
        print("Error, {} es etiquetado en train como {} y en test como {}".format(asientos, label_train, label_test))
        correcto = False
        break
        
if correcto:
    print("Todo perfecto")

Todo perfecto


In [28]:
y = car_train['Precio_cat']
x = car_train.drop(['Precio_cat'],axis=1)

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
x_train,x_test,y_train,y_test = train_test_split(x, y, random_state=0, test_size=0.2)

In [31]:
from sklearn.model_selection import cross_val_score

In [32]:
from sklearn.linear_model import LinearRegression, LogisticRegression

In [33]:
from sklearn.metrics import r2_score

In [34]:
linear_r = LinearRegression()

In [35]:
linear_score = cross_val_score(linear_r, x_train, y_train, cv=5)

In [36]:
print(linear_score.mean(), linear_score.std())

0.6754656580239766 0.026383420222709328


In [54]:
linear_r.fit(x_train,y_train)
linear_pred=linear_r.predict(x_test)
print(linear_pred)
linear_score=r2_score(y_test,linear_pred)

[3.11457793 2.90128056 3.02599096 3.79678189 2.42608953 4.20621273
 4.07897924 3.72846341 4.72482869 3.53201458 4.87457824 2.58934106
 4.21222524 4.21459387 3.2231358  4.53832424 3.11927203 3.41535925
 3.28564392 2.6306398  3.61217587 3.82077493 3.31686411 1.3702326
 3.84904484 3.05957253 3.084458   3.29572061 3.49448275 2.90868827
 3.07277928 3.02160358 3.13955546 2.46745629 3.55767402 3.13026932
 2.57300365 3.41565963 2.28354384 4.20538357 2.74229073 4.33862084
 1.32747211 2.31284154 2.3001815  5.04989715 3.0490221  1.86743058
 4.25734101 2.68014985 2.82371377 1.6105057  3.45887661 3.00071439
 3.27369344 3.99686506 4.08511397 2.94874651 3.72727635 3.53647909
 3.81765084 3.21133091 2.54055185 3.49897855 2.45854799 4.52332011
 5.20872109 2.43920042 2.38660709 3.88822399 3.03633314 2.06276097
 5.99842538 4.86814993 4.5535511  3.04505875 3.24951024 3.65600693
 3.19023208 2.99112309 3.32030389 1.98637904 3.53590177 4.79500482
 4.12366631 2.02318385 3.33643055 2.40551428 2.91834588 3.39861

In [38]:
print(linear_score)

0.7004961000618063


In [39]:
lr = LogisticRegression()

In [40]:
lr_score = cross_val_score(lr, x_train, y_train, cv=5)

/Users/davidalbertomartinvela/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/davidalbertomartinvela/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

In [41]:
print(lr_score.mean(), lr_score.std())

0.471875 0.016741369044973597


In [53]:
lr.fit(x_train,y_train)
lr_pred=lr.predict(x_test)
print(lr_pred)
lr_score=r2_score(y_test,lr_pred)

[3 3 3 3 3 3 3 5 3 3 5 3 3 3 3 5 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 5 3 3 3 3 3 3 3 3 3 5 3 3 3 3 3 3 3 5
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 5 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 5 3 3 3 3 3 3 3 3 3 5 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 5 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 5 3 3 3 3 3 3 3 3 3 3 3 3 5 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 5 3 3 3 3 3 3 3 3 3 3 3 3 3 3 5 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 5 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 5 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 5 3 3 3 3 3 5 3 3 3 3 3 3 3 3 3 5 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 5
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 5 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 

/Users/davidalbertomartinvela/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [42]:
from sklearn.ensemble import RandomForestRegressor

In [43]:
rfg = RandomForestRegressor()

In [44]:
rfg_score = cross_val_score(rfg, x_train, y_train, cv=5)

In [45]:
print(rfg_score.mean(), rfg_score.std())

0.8340734009181837 0.024648736424882342


In [46]:
rfg.fit(x_train,y_train)
rfg_pred=rfg.predict(x_test)
print(rfg_pred)
rfg_score=r2_score(y_test,rfg_pred)

In [47]:
print(rfg_score)

0.8645720627503338


Versión 1 (Vanilla)

Generar fichero

In [58]:
# Ahora predecimos
predict = rfg.predict(car_test)
# Generamos 
df_result = pd.DataFrame({'id': tests_ids, 'Precio_cat': np.around(predict).astype(int) })
df_result.to_csv("mis_resultados_1.csv", index=False)

In [60]:
df_result.head()

,id,Precio_cat
0,4820,5
1,4821,2
2,4822,2
3,4823,3
4,4824,2
